In [1]:
import numpy as np
import torch
# BertModel -> Linear -> GELU -> Linear
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [2]:
model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
text = "今日は [MASK] へ行く。"
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


In [6]:
input_ids = tokenizer.encode(text, return_tensors="pt")
input_ids = input_ids.cuda()

In [7]:
with torch.no_grad():
    output = bert_mlm(input_ids)
    # サイズは(バッチサイズ、系列長、語彙サイズ)
    scores = output.logits

In [10]:
# MASK のidは4
list(tokenizer.vocab.items())[:10]

[('[PAD]', 0),
 ('[UNK]', 1),
 ('[CLS]', 2),
 ('[SEP]', 3),
 ('[MASK]', 4),
 ('の', 5),
 ('、', 6),
 ('に', 7),
 ('。', 8),
 ('は', 9)]

In [14]:
# MASKの位置
mask_position = input_ids[0].tolist().index(4)
# スコアが最も良いトークンのIDを取得
# argmax()で取得した最大値のインデックスが、IDである（語彙方向のインデクスがそのままIDである）
id_best = scores[0, mask_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace("##", "")

text = text.replace("[MASK]", token_best)
print(text)

今日は 東京 へ行く。


In [16]:
torch.Tensor(np.array([10, 1, 9, 2, 8, 3])).topk(3)

torch.return_types.topk(
values=tensor([10.,  9.,  8.]),
indices=tensor([0, 2, 4]))

In [18]:
import numpy.typing as npt

def predict_mask_topk(
    text: str,
    tokenizer: BertJapaneseTokenizer,
    bert_mlm: BertForMaskedLM,
    num_topk: int = 10,
) -> tuple[list[str], npt.NDArray]:
    """文章中最初の[MASK]をスコア上位のトークンに置き換える

    出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト
    """
    # 文章を符号化し、BERTで分類スコアを得る
    input_ids: torch.Tensor = tokenizer.encode(text, return_tensors="pt")
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids)
    # サイズは(バッチサイズ、系列長、語彙サイズ)
    scores = output.logits

    # スコアが上位のトークンとスコアを求める
    mask_position = input_ids[0].tolist().index(4)
    topk: torch.return_types.topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)
    scores_topk = topk.values.cpu().numpy()

    # [MASK]を上で求めたトークンで置換
    text_topk: list[str] = []
    for token in tokens_topk:
        token = token.replace("##", "")
        # 1: 最初のMASKだけ置換
        text_topk.append(text.replace("[MASK]", token, 1))

    return text_topk, scores_topk


text = "今日は[MASK]へ行く。"
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm)
print(*text_topk, sep="\n")

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


In [19]:
def greedy_prediction(
    text: str,
    tokenizer: BertJapaneseTokenizer,
    bert_mlm: BertForMaskedLM,
) -> str:
    """貪欲法で穴埋めを行う"""
    for _ in range(text.count("[MASK]")):
        text = predict_mask_topk(text, tokenizer, bert_mlm, num_topk=1)[0][0]
    return text


text = "今日は[MASK][MASK]へ行く。"
greedy_prediction(text, tokenizer, bert_mlm)

'今日は、東京へ行く。'